In [43]:
import pandas as pd
import numpy as np

In [44]:
data = pd.read_csv('reglementation-des-peches-cartographiee.csv')

In [45]:
data.sample(10)

,type_de_reglementation,thematique,zone,reglementations,wkt
759,Reg. NAMO,Bretagne - Filets,Zone C - tampon 5 milles nautiques,Arrêté Préfectoral R53-2021-10-19-00003 - déli...,"MULTIPOLYGON(((-4.553253 48.294792399985,-4.55..."
10,Reg. NAMO,Auray-Vannes - CSJ - Dragues,A4 - zone soumise à la détention d'une licence...,Arrêté Préfectoral R53-2020-05-19-001 - délib ...,"MULTIPOLYGON(((-3.06884999999999 47.35258,-3.0..."
236,Reg. MEMN,Normandie - Crustacés - Casiers et Filets,Zone à Etrilles : Ravenoville,Arrêté Préfectoral 174/2019 - Délib 2019/28-CR...,"MULTIPOLYGON(((-1.23702454 49.46860132,-1.2236..."
789,Reg. MED,Corse - Chaluts,3 - 12 MN: Autorisation chalut à panneaux,Arrêté Préfectoral 2013070-0001 / MED,MULTIPOLYGON(((9.40545622982063 43.07767545346...
735,Reg. MED,Méditerannée - Chaluts,Eau du littoral du Var et des Alpes-Maritimes,Arrêté Préfectoral 99-162 / MED,MULTIPOLYGON(((5.6480657579954 43.119893331304...
398,Reg. MEMN,Normandie - Crustacés - Casiers et Filets,Zone à Etrilles : Pirou,Arrêté Préfectoral 174/2019 - Délib 2019/28-CR...,"MULTIPOLYGON(((-1.63441323636364 49.1625,-1.63..."
161,Reg. MED,Provence Alpes Côte d'Azur - Cantonnements de ...,Cantonnement du Cap-Couronne,Arrêté Ministériel du 12/02/1998,MULTIPOLYGON(((5.05155763480587 43.32314203337...
462,Reg. MED,Provence Alpes Côte d'Azur - Anguille,Interdictions temporaires pêche Anguille jaune,"Arrêté Préfectoral R93-2022-01-03-00007 / MED,...",MULTIPOLYGON(((4.62101922802863 43.35894679549...
44,Reg. NAMO,Ille-et-Vilaine - Bulots - Casiers,Saint-Malo Ouest - pêche sous licence,Arrêté Préfectoral 2018-16631 - Délib 2018-058...,MULTIPOLYGON(((-2.18904034757802 48.6342799535...
9,Reg. NAMO,Auray-Vannes - CSJ - Dragues,A3 - zone soumise à la détention d'une licence...,Arrêté Préfectoral R53-2020-05-19-001 - délib ...,"MULTIPOLYGON(((-3.12637999999999 47.39427,-3.1..."


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882 entries, 0 to 881
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   type_de_reglementation  882 non-null    object
 1   thematique              882 non-null    object
 2   zone                    879 non-null    object
 3   reglementations         879 non-null    object
 4   wkt                     882 non-null    object
dtypes: object(5)
memory usage: 34.6+ KB


In [47]:
data[data.zone.isna()]

,type_de_reglementation,thematique,zone,reglementations,wkt
813,Reg. ORGP,OPANO/NAFO - Crevette nordique,NaN,NaN,"MULTIPOLYGON(((-45.6666666666667 47.5,-45 47.9..."
814,Reg. ORGP,OPANO/NAFO - Pêche de fond,NaN,NaN,MULTIPOLYGON(((-45.0083333333333 50.0083333333...
815,Reg. ORGP,OPANO/NAFO - Crevette nordique,NaN,NaN,MULTIPOLYGON(((-41.9999999999999 49.2500000000...


In [48]:
data.describe(include="all")

,type_de_reglementation,thematique,zone,reglementations,wkt
count,882,882,879,879,882
unique,11,228,858,295,836
top,Reg. NAMO,Civelle,Zone d'interdiction de pêche,R(UE) 2019/1241,"MULTIPOLYGON(((3.54670758 43.3760237,3.5470558..."
freq,331,26,4,55,10


In [49]:
data.type_de_reglementation.value_counts()

Reg. NAMO          331
Reg. MED           166
Reg. MEMN          152
Reg. Outre-mer      90
R(UE) 2019/1241     76
Reg. SA             49
R(UE) 1380/2013      8
Reg. ORGP            3
R(UE) 2022/1614      3
R(CE) 494/2002       2
Reg. CCAMLR          2
Name: type_de_reglementation, dtype: int64

In [50]:
data.reglementations.value_counts()

R(UE) 2019/1241                                                                                                                     55
Arrêté Préfectoral 96/DRAM/2077 / NAMO                                                                                              26
Arrêté Préfectoral 412/2008 / MED                                                                                                   23
Arrêté Préfectoral R53-2020-05-19-001 - délib 2020-006 / NAMO, Arrêté Préfectoral R53-2019-09-12-009 / Délib 2019-023 / NAMO CSJ    22
R(UE) 2019/1241 Modifié                                                                                                             21
                                                                                                                                    ..
Arrêté Préfectoral du 30/12/2014                                                                                                     1
Arrêté 73 du 04/07/1979 / NAMO, Arrêté Préfectoral R53-

In [51]:
data.zone.value_counts()

Zone d'interdiction de pêche       4
Zone réglementée 2                 3
Zone réglementée 1                 3
Baie de Bourgneuf                  3
Zone autorisée - Seine-Maritime    2
                                  ..
Coques Vilaine Dragues             1
Box merlu - Irlande                1
Box merlu - Golfe de Gascogne      1
Filet maillant et trémail          1
Zone E : réservée arts dormants    1
Name: zone, Length: 858, dtype: int64

In [52]:
data[data.zone == "Zone réglementée 2"]

,type_de_reglementation,thematique,zone,reglementations,wkt
378,Reg. MEMN,Ouest Cotentin - Dragues - Coquille St Jacques,Zone réglementée 2,Arrêté Préfectoral 147/2022 - délib 2022/E-CSJ...,MULTIPOLYGON(((-2.08333333333333 49.3542386019...
604,Reg. Outre-mer,Saint Pierre et Miquelon - Concombre de mer,Zone réglementée 2,Arrêté Ministériel du 31/05/2022,MULTIPOLYGON(((-56.7988455095294 46.5219413816...
607,Reg. Outre-mer,Saint Pierre et Miquelon - Crabe des neiges,Zone réglementée 2,Arrêté Ministériel du 01/04/2022 / Saint-Pierr...,MULTIPOLYGON(((-56.1517110414139 46.0000460614...


In [53]:
data[data.zone == "Coques Vilaine Dragues"]

,type_de_reglementation,thematique,zone,reglementations,wkt
292,Reg. NAMO,Morbihan - bivalves,Coques Vilaine Dragues,Arrêté Préfectoral 2016-13780 - délib 2016-056...,"MULTIPOLYGON(((-2.51020277777778 47.49575,-2.5..."


In [63]:
zones_interdites = []
for zone in data.zone.unique():
    zone = str(zone)
    if zone.find("interdi") != -1:
        zones_interdites.append(zone)

In [65]:
len(zones_interdites)

93

In [66]:
zones_interdites

['Banc de Maërl - Zone interdite à la pêche',
 'Banc de Maërl - interdiction de pêche',
 'Herbiers de zostères "Baie de Morlaix" - interdiction',
 "Kerfinel - zone d'interdiction temporaire",
 'Zone de cohabitation 2 - interdiction pêche',
 'Zone de cohabitation 1 - pêche interdite',
 'Eaux territoriales Bretagne - Pêche interdite',
 'Zone A - interdiction chalutage',
 'Zone B - interdiction chalutage',
 'Nord Groix - Zone interdite',
 'Sud Groix - Zone interdite',
 'Belle-Île - interdiction temporaire',
 'Quiberon - Zone interdite temporaire',
 "Zone d'interdiction : Ile-verte",
 "Zone d'interdiction : Epine-Est",
 "Zone d'interdiction : Agde",
 "Zone d'interdiction : Palavas",
 "Zone d'interdiction : Gruissan",
 "Zone d'interdiction : Fleury D'Aude",
 "Hérault et Gard - Pêche sous-marine interdite - Grau d'Adge",
 'Hérault et Gard - Pêche sous-marine interdite - Etang de Thau',
 "Zone d'interdiction 1 : Marseillan-Sète",
 "Zone d'interdiction 2 : Marseillan-Sète",
 "Zone d'interdicti